In [4]:
!pip install pandas requests


Defaulting to user installation because normal site-packages is not writeable


In [28]:
 import requests
import pandas as pd

# LeetCode GraphQL API URL
GRAPHQL_URL = "https://leetcode.com/graphql/"
HEADERS = {
    "Content-Type": "application/json",
    "User-Agent": "Mozilla/5.0"
}

def fetch_leetcode_data(username):
    """
    Fetches LeetCode profile data for a given username.
    """
    # GraphQL queries
    ranking_query = """
    query userContestRankingInfo($username: String!) {
      userContestRanking(username: $username) {
        rating
      }
    }
    """
    
    problems_query = """
    query userProfileUserQuestionProgressV2($userSlug: String!) {
      userProfileUserQuestionProgressV2(userSlug: $userSlug) {
        numAcceptedQuestions {
          count
        }
      }
    }
    """
    
    badges_query = """
    query userBadges($username: String!) {
      matchedUser(username: $username) {
        badges {
          displayName
        }
      }
    }
    """
    
    # Fetch data for rating
    try:
        ranking_response = requests.post(GRAPHQL_URL, json={"query": ranking_query, "variables": {"username": username}}, headers=HEADERS)
        
        if ranking_response.status_code == 200:
            response_data = ranking_response.json()
            rating = response_data.get("data", {}).get("userContestRanking", {}).get("rating", "N/A")
        else:
            rating = "N/A"
    except Exception as e:
        rating = "N/A"
    
    # Fetch data for problems solved
    try:
        problems_response = requests.post(GRAPHQL_URL, json={"query": problems_query, "variables": {"userSlug": username}}, headers=HEADERS)
        if problems_response.status_code == 200:
            problems_data = problems_response.json().get("data", {}).get("userProfileUserQuestionProgressV2", {}).get("numAcceptedQuestions", [])
            total_problems_solved = sum(item["count"] for item in problems_data) if problems_data else 0
        else:
            total_problems_solved = 0
    except Exception as e:
        total_problems_solved = 0
    
    # Fetch data for badges
    try:
        badges_response = requests.post(GRAPHQL_URL, json={"query": badges_query, "variables": {"username": username}}, headers=HEADERS)
        if badges_response.status_code == 200:
            badges = badges_response.json().get("data", {}).get("matchedUser", {}).get("badges", [])
            badges_display = ", ".join(badge["displayName"] for badge in badges) if badges else "No badges"
        else:
            badges_display = "No badges"
    except Exception as e:
        badges_display = "No badges"
    
    # Combine the data into a dictionary
    return {
        "Username": username,
        "Rating": rating,
        "Total Problems Solved": total_problems_solved,
        "Badges": badges_display
    }

def read_usernames(file_path):
    """
    Reads usernames from a file. Supports CSV and plain text files.
    :param file_path: Path to the file containing usernames
    :return: List of usernames
    """
    try:
        if file_path.endswith(".csv"):
            df = pd.read_csv(file_path)
            return df.iloc[:, 0].tolist()  # Assumes usernames are in the first column
        else:
            with open(file_path, "r") as file:
                return [line.strip() for line in file.readlines()]
    except FileNotFoundError:
        return []

def fetch_bulk_users(usernames):
    """
    Fetches LeetCode profile data for multiple users and combines it into a list of dictionaries.
    :param usernames: List of usernames
    :return: List of dictionaries containing user profile data
    """
    results = []
    for username in usernames:
        results.append(fetch_leetcode_data(username))
    return results

if __name__ == "__main__":
    # Input file containing usernames (CSV or plain text file)
    file_path = input("Enter the path to the file containing usernames: ").strip()

    # Read usernames from the file
    usernames = read_usernames(file_path)

    # Fetch data for all users
    data = fetch_bulk_users(usernames)

    # Convert to a DataFrame for display
    df = pd.DataFrame(data)

    # Display the results in a table
    print("\n--- LeetCode Profiles ---")
    print(df)

    # Save to CSV
    output_file = "leetcode_profiles_minimal.csv"
    df.to_csv(output_file, index=False)
    print(f"\nData saved to '{output_file}'")


Enter the path to the file containing usernames:  C:\Users\sowja\Downloads\usernames.txt



--- LeetCode Profiles ---
                                 Username       Rating  Total Problems Solved  \
0                           sowjanyakodam          N/A                    176   
1                               Veekshana     1412.278                    132   
2                               Pavan1729          N/A                    128   
3                         Aravind_kammari     1339.408                    418   
4                          bharathganeshn          N/A                    143   
5                        vigneshwar_banda          N/A                     89   
6                          Sathwika_Gundu          N/A                    173   
7                           Gollashivanii          N/A                     94   
8                                himateja          N/A                      9   
9                              bkiransree     1510.757                     78   
10                               Bagga_10          N/A                     83   
1